# Unit 3: Basic Data Inspection - Practical Exercises

## Table of Contents
1. [Introduction](#introduction)
2. [Setup](#setup)
3. [Using .info()](#using-info)
4. [Using .describe()](#using-describe)
5. [Detecting Missing Values](#detecting-missing-values)
6. [Counting Unique Values](#counting-unique-values)
7. [Frequency Analysis with value_counts()](#frequency-analysis)
8. [Complete Inspection Workflow](#complete-inspection-workflow)
9. [Practice with Multiple Datasets](#practice-multiple-datasets)

---

## 1. Introduction <a id='introduction'></a>

In this practical notebook, you'll learn to systematically inspect datasets using pandas' essential functions. These skills are critical for understanding your data before any analysis.

**Learning Objectives:**
- Master `.info()`, `.describe()`, `.isnull()`, `.nunique()`, and `.value_counts()`
- Learn to identify data quality issues
- Develop a systematic inspection workflow
- Practice with real-world datasets

## 2. Setup <a id='setup'></a>

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# Settings
%matplotlib inline
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 2)

print("✅ Setup complete!")
print(f"pandas version: {pd.__version__}")
print(f"numpy version: {np.__version__}")

## 3. Using .info() <a id='using-info'></a>

The `.info()` method provides a concise summary of your DataFrame.

In [ ]:
# Load Titanic dataset
titanic = sns.load_dataset('titanic')

print("TITANIC DATASET - .info() ANALYSIS")
print("="*80)
titanic.info()

print("\n" + "="*80)
print("KEY OBSERVATIONS FROM .info():")
print("="*80)
print(f"Total entries (rows): {len(titanic)}")
print(f"Total columns: {len(titanic.columns)}")
print(f"\nMemory usage: {titanic.memory_usage(deep=True).sum() / 1024:.2f} KB")

In [ ]:
# Analyzing data types
print("DATA TYPES BREAKDOWN")
print("="*80)
print(titanic.dtypes)

print("\n" + "="*80)
print("Count by Data Type:")
print(titanic.dtypes.value_counts())

# Identify numerical and categorical columns
numerical_cols = titanic.select_dtypes(include=[np.number]).columns.tolist()
categorical_cols = titanic.select_dtypes(include=['object', 'category']).columns.tolist()

print("\n" + "="*80)
print(f"Numerical columns ({len(numerical_cols)}):")
print(numerical_cols)

print(f"\nCategorical columns ({len(categorical_cols)}):")
print(categorical_cols)

In [ ]:
# Detecting columns with missing values from .info()
print("COLUMNS WITH MISSING DATA (from .info())")
print("="*80)

total_rows = len(titanic)
print(f"Total rows: {total_rows}\n")

for col in titanic.columns:
    non_null = titanic[col].count()
    if non_null < total_rows:
        missing = total_rows - non_null
        missing_pct = (missing / total_rows) * 100
        print(f"{col:15} - Missing: {missing:4d} ({missing_pct:5.2f}%)")

## 4. Using .describe() <a id='using-describe'></a>

The `.describe()` method generates statistical summaries.

In [ ]:
# Basic describe for numerical columns
print("STATISTICAL SUMMARY - Numerical Columns")
print("="*80)
titanic.describe()

In [ ]:
# Describe all columns (including categorical)
print("STATISTICAL SUMMARY - All Columns")
print("="*80)
titanic.describe(include='all')

In [ ]:
# Analyzing specific columns in detail
print("DETAILED ANALYSIS: Age Column")
print("="*80)

age_stats = titanic['age'].describe()
print(age_stats)

print("\n" + "="*80)
print("INTERPRETATION:")
print("="*80)
print(f"Count: {age_stats['count']:.0f} non-null values out of {len(titanic)} total")
print(f"Mean age: {age_stats['mean']:.1f} years")
print(f"Median age: {age_stats['50%']:.1f} years")
print(f"Youngest passenger: {age_stats['min']:.1f} years")
print(f"Oldest passenger: {age_stats['max']:.1f} years")
print(f"Standard deviation: {age_stats['std']:.1f} years")

# Check for skewness
mean_median_diff = abs(age_stats['mean'] - age_stats['50%'])
if mean_median_diff < 1:
    print("\n✅ Distribution appears relatively symmetric (mean ≈ median)")
elif age_stats['mean'] > age_stats['50%']:
    print("\n⚠️ Distribution is right-skewed (mean > median)")
else:
    print("\n⚠️ Distribution is left-skewed (mean < median)")

In [ ]:
# Visualize the distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Histogram
titanic['age'].hist(bins=30, ax=ax1, edgecolor='black')
ax1.axvline(age_stats['mean'], color='red', linestyle='--', label=f"Mean: {age_stats['mean']:.1f}")
ax1.axvline(age_stats['50%'], color='green', linestyle='--', label=f"Median: {age_stats['50%']:.1f}")
ax1.set_xlabel('Age')
ax1.set_ylabel('Frequency')
ax1.set_title('Age Distribution - Histogram')
ax1.legend()

# Box plot
titanic.boxplot(column='age', ax=ax2)
ax2.set_ylabel('Age')
ax2.set_title('Age Distribution - Box Plot')

plt.tight_layout()
plt.show()

In [ ]:
# Identify potential outliers
print("OUTLIER DETECTION using IQR Method")
print("="*80)

Q1 = titanic['age'].quantile(0.25)
Q3 = titanic['age'].quantile(0.75)
IQR = Q3 - Q1

lower_bound = Q1 - 1.5 * IQR
upper_bound = Q3 + 1.5 * IQR

print(f"Q1 (25th percentile): {Q1:.1f}")
print(f"Q3 (75th percentile): {Q3:.1f}")
print(f"IQR: {IQR:.1f}")
print(f"\nOutlier bounds:")
print(f"  Lower bound: {lower_bound:.1f}")
print(f"  Upper bound: {upper_bound:.1f}")

outliers = titanic[(titanic['age'] < lower_bound) | (titanic['age'] > upper_bound)]
print(f"\nNumber of outliers: {len(outliers)}")
print(f"Percentage: {len(outliers)/len(titanic)*100:.2f}%")

if len(outliers) > 0:
    print(f"\nOutlier ages: {sorted(outliers['age'].dropna().unique())}")

## 5. Detecting Missing Values <a id='detecting-missing-values'></a>

Missing values can significantly impact your analysis.

In [ ]:
# Count missing values per column
print("MISSING VALUES ANALYSIS")
print("="*80)

missing_count = titanic.isnull().sum()
missing_pct = (missing_count / len(titanic) * 100).round(2)

missing_df = pd.DataFrame({
    'Column': titanic.columns,
    'Missing Count': missing_count.values,
    'Percentage': missing_pct.values
})

# Show only columns with missing values
missing_df_filtered = missing_df[missing_df['Missing Count'] > 0].sort_values('Percentage', ascending=False)

if len(missing_df_filtered) > 0:
    print("\nColumns with Missing Values:")
    print(missing_df_filtered.to_string(index=False))
else:
    print("\n✅ No missing values found!")

print("\n" + "="*80)
print("SUMMARY:")
print(f"Total cells: {titanic.shape[0] * titanic.shape[1]}")
print(f"Missing cells: {titanic.isnull().sum().sum()}")
print(f"Percentage missing: {(titanic.isnull().sum().sum() / (titanic.shape[0] * titanic.shape[1]) * 100):.2f}%")

In [ ]:
# Visualize missing data
plt.figure(figsize=(12, 6))
sns.heatmap(titanic.isnull(), cbar=False, yticklabels=False, cmap='viridis')
plt.title('Missing Values Heatmap (Yellow = Missing)', fontsize=14)
plt.xlabel('Columns')
plt.ylabel('Rows')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()
plt.show()

In [ ]:
# Missing value patterns
print("MISSING VALUE PATTERNS")
print("="*80)

# Check if missing values are correlated
print("\nRows with multiple missing values:")
rows_with_missing = titanic[titanic.isnull().any(axis=1)]
missing_per_row = rows_with_missing.isnull().sum(axis=1)
print(f"Total rows with at least one missing value: {len(rows_with_missing)}")
print(f"\nDistribution of missing values per row:")
print(missing_per_row.value_counts().sort_index())

# Check if age and cabin are both missing
both_missing = titanic[titanic['age'].isnull() & titanic['deck'].isnull()]
print(f"\nRows where both age AND deck are missing: {len(both_missing)}")

In [ ]:
# Missing value bar chart
missing_data = titanic.isnull().sum()
missing_data = missing_data[missing_data > 0].sort_values(ascending=False)

if len(missing_data) > 0:
    plt.figure(figsize=(10, 6))
    missing_data.plot(kind='bar', color='coral', edgecolor='black')
    plt.title('Missing Values by Column', fontsize=14)
    plt.xlabel('Column')
    plt.ylabel('Number of Missing Values')
    plt.xticks(rotation=45, ha='right')
    plt.grid(axis='y', alpha=0.3)
    plt.tight_layout()
    plt.show()

## 6. Counting Unique Values <a id='counting-unique-values'></a>

Understanding the number of unique values helps identify categorical vs numerical columns.

In [ ]:
# Count unique values per column
print("UNIQUE VALUES PER COLUMN")
print("="*80)

unique_counts = titanic.nunique().sort_values()
print(unique_counts)

print("\n" + "="*80)
print("CATEGORIZATION BY UNIQUE COUNT:")
print("="*80)

binary_cols = unique_counts[unique_counts == 2].index.tolist()
low_cardinality = unique_counts[(unique_counts > 2) & (unique_counts <= 10)].index.tolist()
medium_cardinality = unique_counts[(unique_counts > 10) & (unique_counts <= 50)].index.tolist()
high_cardinality = unique_counts[unique_counts > 50].index.tolist()

print(f"\nBinary columns (2 unique): {binary_cols}")
print(f"Low cardinality (3-10 unique): {low_cardinality}")
print(f"Medium cardinality (11-50 unique): {medium_cardinality}")
print(f"High cardinality (>50 unique): {high_cardinality}")

In [ ]:
# Visualize unique counts
plt.figure(figsize=(12, 6))
unique_counts.plot(kind='barh', color='skyblue', edgecolor='black')
plt.title('Number of Unique Values per Column', fontsize=14)
plt.xlabel('Number of Unique Values')
plt.ylabel('Column')
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Examine unique values for low cardinality columns
print("UNIQUE VALUES FOR LOW CARDINALITY COLUMNS")
print("="*80)

for col in low_cardinality + binary_cols:
    print(f"\n{col.upper()}:")
    print(f"Number of unique values: {titanic[col].nunique()}")
    unique_vals = titanic[col].unique()
    # Convert all unique values to string for safe sorting
    unique_vals_str = sorted([str(x) for x in unique_vals])
    print(f"Unique values: {unique_vals_str}")

## 7. Frequency Analysis with value_counts() <a id='frequency-analysis'></a>

Analyze the distribution of categorical variables.

In [ ]:
# Survival distribution
print("SURVIVAL DISTRIBUTION")
print("="*80)

survival_counts = titanic['survived'].value_counts()
survival_pct = titanic['survived'].value_counts(normalize=True) * 100

print("\nCounts:")
print(survival_counts)

print("\nPercentages:")
print(survival_pct.round(2))

print("\n" + "="*80)
print(f"Survival rate: {survival_pct[1]:.2f}%")
print(f"Death rate: {survival_pct[0]:.2f}%")

In [ ]:
# Visualize survival distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

# Bar plot
survival_counts.plot(kind='bar', ax=ax1, color=['red', 'green'], edgecolor='black')
ax1.set_title('Survival Count', fontsize=14)
ax1.set_xlabel('Survived (0=No, 1=Yes)')
ax1.set_ylabel('Count')
ax1.set_xticklabels(['Did Not Survive', 'Survived'], rotation=0)

# Pie chart
survival_counts.plot(kind='pie', ax=ax2, autopct='%1.1f%%', colors=['red', 'green'])
ax2.set_title('Survival Percentage', fontsize=14)
ax2.set_ylabel('')
ax2.legend(['Did Not Survive', 'Survived'], loc='best')

plt.tight_layout()
plt.show()

In [ ]:
# Passenger class distribution
print("PASSENGER CLASS DISTRIBUTION")
print("="*80)

class_counts = titanic['pclass'].value_counts().sort_index()
class_pct = titanic['pclass'].value_counts(normalize=True).sort_index() * 100

class_df = pd.DataFrame({
    'Class': class_counts.index,
    'Count': class_counts.values,
    'Percentage': class_pct.values.round(2)
})

print(class_df.to_string(index=False))

# Visualize
plt.figure(figsize=(10, 6))
class_counts.plot(kind='bar', color=['gold', 'silver', '#CD7F32'], edgecolor='black')
plt.title('Passenger Distribution by Class', fontsize=14)
plt.xlabel('Passenger Class')
plt.ylabel('Count')
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

In [ ]:
# Gender distribution
print("GENDER DISTRIBUTION")
print("="*80)

gender_counts = titanic['sex'].value_counts()
gender_pct = titanic['sex'].value_counts(normalize=True) * 100

gender_df = pd.DataFrame({
    'Gender': gender_counts.index,
    'Count': gender_counts.values,
    'Percentage': gender_pct.values.round(2)
})

print(gender_df.to_string(index=False))

In [ ]:
# Embarkation port distribution
print("EMBARKATION PORT DISTRIBUTION")
print("="*80)

embarked_counts = titanic['embarked'].value_counts(dropna=False)
print("\nCounts (including missing):")
print(embarked_counts)

print("\n" + "="*80)
print("Port Legend:")
print("  S = Southampton")
print("  C = Cherbourg")
print("  Q = Queenstown")
print("  NaN = Missing/Unknown")

## 8. Complete Inspection Workflow <a id='complete-inspection-workflow'></a>

Let's apply a systematic inspection workflow to a new dataset.

In [ ]:
# Load a new dataset
tips = sns.load_dataset('tips')

print("="*80)
print("COMPLETE DATA INSPECTION WORKFLOW - TIPS DATASET")
print("="*80)

In [ ]:
# STEP 1: Basic Structure
print("\n" + "="*80)
print("STEP 1: BASIC STRUCTURE")
print("="*80)

print(f"Shape: {tips.shape} (rows, columns)")
print(f"Columns: {tips.columns.tolist()}")

print("\nFirst 5 rows:")
display(tips.head())

print("\nLast 3 rows:")
display(tips.tail(3))

In [ ]:
# STEP 2: Data Types and Info
print("\n" + "="*80)
print("STEP 2: DATA TYPES AND INFO")
print("="*80)

tips.info()

print("\nData Type Summary:")
print(tips.dtypes.value_counts())

In [ ]:
# STEP 3: Statistical Summary
print("\n" + "="*80)
print("STEP 3: STATISTICAL SUMMARY")
print("="*80)

print("\nNumerical columns:")
display(tips.describe())

print("\nAll columns:")
display(tips.describe(include='all'))

In [ ]:
# STEP 4: Missing Values
print("\n" + "="*80)
print("STEP 4: MISSING VALUES")
print("="*80)

missing = tips.isnull().sum()
if missing.sum() == 0:
    print("\n✅ No missing values detected!")
else:
    missing_pct = (missing / len(tips) * 100).round(2)
    missing_df = pd.DataFrame({
        'Missing Count': missing,
        'Percentage': missing_pct
    })
    print(missing_df[missing_df['Missing Count'] > 0])

In [ ]:
# STEP 5: Unique Values
print("\n" + "="*80)
print("STEP 5: UNIQUE VALUES PER COLUMN")
print("="*80)

print(tips.nunique().sort_values())

In [ ]:
# STEP 6: Categorical Analysis
print("\n" + "="*80)
print("STEP 6: CATEGORICAL COLUMNS ANALYSIS")
print("="*80)

categorical_cols = tips.select_dtypes(include=['object', 'category']).columns

for col in categorical_cols:
    print(f"\n{col.upper()}:")
    print(tips[col].value_counts())
    print(f"\nPercentages:")
    print((tips[col].value_counts(normalize=True) * 100).round(2))
    print("-" * 40)

In [ ]:
# STEP 7: Duplicates
print("\n" + "="*80)
print("STEP 7: DUPLICATE ROWS")
print("="*80)

duplicates = tips.duplicated().sum()
print(f"Number of duplicate rows: {duplicates}")

if duplicates > 0:
    print(f"Percentage: {(duplicates / len(tips) * 100):.2f}%")
    print("\nDuplicate rows:")
    print(tips[tips.duplicated(keep=False)].sort_values(by=tips.columns.tolist()))
else:
    print("\n✅ No duplicate rows found!")

In [ ]:
# STEP 8: Summary Report
print("\n" + "="*80)
print("INSPECTION SUMMARY REPORT")
print("="*80)

print(f"\nDataset: Tips")
print(f"Total rows: {len(tips)}")
print(f"Total columns: {len(tips.columns)}")
print(f"\nColumn Types:")
print(f"  Numerical: {len(tips.select_dtypes(include=[np.number]).columns)}")
print(f"  Categorical: {len(tips.select_dtypes(include=['object', 'category']).columns)}")
print(f"\nData Quality:")
print(f"  Columns with missing data: {(tips.isnull().sum() > 0).sum()}")
print(f"  Duplicate rows: {duplicates}")
print(f"  Memory usage: {tips.memory_usage(deep=True).sum() / 1024:.2f} KB")

print("\n" + "="*80)
print("✅ Inspection complete!")
print("="*80)

## 9. Practice with Multiple Datasets <a id='practice-multiple-datasets'></a>

Apply the inspection workflow to different datasets.

In [ ]:
# Quick inspection function
def quick_inspect(df, name="Dataset"):
    """
    Perform quick inspection of a DataFrame.
    
    Parameters:
    df (DataFrame): The dataset to inspect
    name (str): Name of the dataset
    """
    print("="*80)
    print(f"QUICK INSPECTION: {name.upper()}")
    print("="*80)
    
    print(f"\nShape: {df.shape}")
    print(f"Memory: {df.memory_usage(deep=True).sum() / 1024:.2f} KB")
    
    print("\nMissing Values:")
    missing = df.isnull().sum()
    if missing.sum() == 0:
        print("  ✅ None")
    else:
        missing_cols = missing[missing > 0]
        for col, count in missing_cols.items():
            pct = (count / len(df)) * 100
            print(f"  {col}: {count} ({pct:.2f}%)")
    
    print(f"\nDuplicates: {df.duplicated().sum()}")
    
    print("\nData Types:")
    print(f"  Numerical: {len(df.select_dtypes(include=[np.number]).columns)}")
    print(f"  Categorical: {len(df.select_dtypes(include=['object', 'category']).columns)}")
    
    print("\n" + "="*80)

# Test the function
quick_inspect(tips, "Tips")

In [ ]:
# Inspect multiple datasets
datasets = {
    'Iris': sns.load_dataset('iris'),
    'Diamonds': sns.load_dataset('diamonds'),
    'Penguins': sns.load_dataset('penguins')
}

for name, df in datasets.items():
    quick_inspect(df, name)
    print("\n")

## Summary

In this practical notebook, you've learned:

✅ **Using .info()** for comprehensive DataFrame overview

✅ **Using .describe()** for statistical summaries

✅ **Detecting missing values** with .isnull() and .sum()

✅ **Counting unique values** with .nunique()

✅ **Analyzing frequencies** with .value_counts()

✅ **Following a systematic inspection workflow**

✅ **Visualizing data quality issues**

✅ **Creating inspection functions** for reusability

### Key Functions Summary:

| Function | Purpose | Output |
|----------|---------|--------|
| `.info()` | Overview | Prints structure |
| `.describe()` | Statistics | DataFrame |
| `.isnull().sum()` | Missing count | Series |
| `.nunique()` | Unique count | Series |
| `.value_counts()` | Frequency | Series |

---

**Copyright Notice**

© 2026 Exploratory Data Analysis Course. All Rights Reserved.

The materials in this practical notebook are protected by intellectual property, copyright, and education privacy laws. Unauthorized copying, distribution, or adaptation is strictly prohibited.